In [30]:
# import the opencv library
import cv2
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [31]:
# import our model
from keras.models import load_model

In [32]:
# Other dependencies
import numpy as np
import tensorflow as tf
import os
from PIL import Image

In [33]:
# Crop center of the image feed
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

# Crop largest square from the image feed
def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

# Crop image around face
def crop_face(pil_img, x, y, w, h):
    return pil_img.crop((x, y, x+w, y+h))

In [34]:
# define a video capture object
stream = cv2.VideoCapture(0)
model = load_model('emotion_model.h5')

while(True):
    try:
        # Capture the video frame by frame
        res, frame = stream.read()
        height, width, channel = frame.shape
        
        #Convert the captured frame into RGB
        im = Image.fromarray(frame, 'RGB')
        
        # Find face in the frame
        face = face_haar_cascade.detectMultiScale(frame)
        x = face[0][0]
        y = face[0][1]
        w = face[0][2]
        h = face[0][3]
        
        #Rectangle around the face
        cv2.rectangle(img = frame, pt1 = (x,y), pt2 = (x+w, y+h), color = (255, 0, 0), thickness= 2)
        
        # # Cropping image around face
        # im_new = crop_face(im, x-10, y+10, w+10, h+10)
        
        # Resizing into dimensions you used while training
        im_new_new = crop_max_square(im)
        im_new_new = im_new_new.resize((48,48))
        img_array = np.array(im_new_new)
        
        #Expand dimensions to match the 4D Tensor shape.
        img_array = np.expand_dims(img_array, axis=1)
        img_array = img_array.reshape(-1, 48, 48, 1)
        
        # Scale to match train data
        img_array = img_array/255
        
        #Calling the predict function using keras
        prediction = model.predict(img_array)
        print(prediction)
        
        # Taking only the first of three arrays
        array = prediction[0]
        
        # Find location of the highest probability
        percent = round(max(array),2)
        predict = np.where(array == max(array))
        emotion = predict[0]
        
        # Part of the code that predicts the emotion from the prediction array
        if(emotion == 0):
            output = "Angry"
        elif(emotion == 1):
            output = "Disgust"
        elif(emotion == 2):
            output = "Fear"
        elif(emotion == 3):
            output = "Happy"
        elif(emotion == 4):
            output = "Sad"
        elif(emotion == 5):
            output = "Surprise"
        elif(emotion == 6):
            output = "Neutral"
            
        print(output)
        
        cv2.putText(frame, f'{output}', (0,50), cv2.FONT_HERSHEY_DUPLEX, 2, (255,255,255), 2)
        # Display the resulting frame
        cv2.imshow('Predict Emotion', frame)
    
    except :
        pass
    
    # the 'q' button is set as the quitting button
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
stream.release()

# Destroy all the windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 186ms/step
[[0.02269148 0.00303768 0.0120998  0.50081986 0.4247911  0.00082187
  0.0357383 ]
 [0.13498439 0.00225656 0.08525629 0.24550104 0.44199508 0.00155701
  0.08844963]
 [0.10635214 0.18651961 0.04139201 0.38215    0.20757072 0.0045947
  0.07142091]]
Happy
1/1 [==============================] - 0s 34ms/step
[[0.02022596 0.00186607 0.01061007 0.7170199  0.19009957 0.00198028
  0.05819815]
 [0.22208029 0.00191645 0.08508846 0.31547788 0.2428542  0.00534793
  0.12723474]
 [0.09192124 0.27167243 0.03722867 0.3919384  0.15450718 0.00406187
  0.0486702 ]]
Happy
1/1 [==============================] - 0s 40ms/step
[[0.01981123 0.00174922 0.01105697 0.7218834  0.18703215 0.00191744
  0.05654958]
 [0.22747524 0.00196128 0.08388589 0.3063243  0.24598739 0.00542526
  0.12894063]
 [0.0871085  0.27609637 0.03825837 0.393143   0.15096492 0.00403702
  0.05039176]]
Happy
1/1 [==============================] - 0s 27ms/step
[[0.01898709 0.00178888 0.0107620